## 1. Setting up the environment

In [1]:
import numpy as np
import gym

#pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
# setting manual seed
torch.manual_seed(0)

from parallelEnv import parallelEnv

#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# imports for rendering outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

In [2]:
# lets set up the cartpole environment
env = gym.make('Ant-v2')
env.seed(0)

# lets find about the action and state space
print("Action space: {} ... State space: {}".format(env.action_space,env.observation_space))

# lets print some info about action space range
action_high = env.action_space.high
action_low = env.action_space.low
print("action_high: {}\t action_low: {}".format(action_high,action_low))

Action space: Box(8,) ... State space: Box(111,)
action_high: [1. 1. 1. 1. 1. 1. 1. 1.]	 action_low: [-1. -1. -1. -1. -1. -1. -1. -1.]


In [ ]:
# lets play a random episode

# reset env
state = env.reset()
step_count = 0
done = False

while (not done):
    # random action
    action = env.action_space.sample()
    # taking step in env
    next_state,reward,done,_ = env.step(action)
    
    # displaying progress
    print ("Step : {} ... state: {} ... action: {} ... reward: {} ... done: {}".format(step_count,
                                                                                       state[:4],action[:4],reward,done))
    
    # updating state and step count
    state = next_state
    step_count+=1

## 2. Defining the policy

In [ ]:
# defining the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("using",device)

In [ ]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

class Policy(nn.Module):
    
    def __init__(self,state_size,action_size,action_high,action_low,hidden_size=32):
        super(Policy, self).__init__()
        
        # action range
        self.action_high = torch.tensor(action_high).to(device)
        self.action_low = torch.tensor(action_low).to(device)
        
        # fc layers for the policy network
        self.fc1 = nn.Linear(state_size,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3_action = nn.Linear(128,action_size)
        self.fc3_std = nn.Linear(128,action_size)
    
    def forward(self,state):
        net = F.relu(self.fc1(state))
        net = F.relu(self.fc2(net))
        action_mean = F.sigmoid(self.fc3_action(net))
        # rescale action mean
        action_mean_ = (self.action_high-self.action_low)*action_mean + self.action_low
        action_std = F.sigmoid(self.fc3_std(net))
        return action_mean_,action_std
    
    def act(self,state):
        # converting state from numpy array to pytorch tensor on the "device"
        state = torch.from_numpy(state).float().to(device)
        action_mean,action_std = self.forward(state)
        prob_dist = Normal(action_mean,action_std)
        action = prob_dist.sample()
        log_prob = prob_dist.log_prob(action)
        return action.cpu().numpy(),torch.sum(log_prob,dim=1)

## 3. Defining the RL agent

In [ ]:
from collections import deque
from itertools import accumulate

def compute_future_rewards(rewards,gamma):
    future_rewards = np.zeros_like(rewards)
    discounted_rewards = np.zeros(rewards.shape[0])
    
    for time_step in range(future_rewards.shape[1]-1,-1,-1):
        future_rewards[:,time_step] = rewards[:,time_step] + gamma*discounted_rewards
        discounted_rewards = future_rewards[:,time_step]
    return future_rewards

class Agent:
    
    def __init__(self,env_name,learning_rate=1e-3):
        self.env = parallelEnv(env_name=env_name,n=8,seed=0)
        self.nS = env.observation_space.shape[0]
        self.nA = env.action_space.shape[0]
        self.policy = Policy(state_size=self.nS,hidden_size=128,action_size=self.nA,
                             action_low=action_low,action_high=action_high).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
        
    def train(self,max_opt_steps=1000,num_trajectories=8,horizon=1000,gamma=.99,PRINT_EVERY=100):
        # store eps scores
        scores = []
        scores_window = deque(maxlen=100)
        
        for opt_step in range(1,max_opt_steps+1):
            rewards = np.zeros([num_trajectories,horizon])
            log_probs = torch.zeros([num_trajectories,horizon])
            
            for traj_count in range(1):
                # reset state
                state = self.env.reset()
            
                # play an episode
                for t in range(horizon): 
                    action,log_prob = self.policy.act(state)
                    next_state,reward,done,_ = self.env.step(action)

                    # update state
                    state = next_state
                    log_probs[:,t] = log_prob
                    rewards[:,t] = reward

                    # break if done
                    if np.any(done):
                        break
            
            # compute advantage estimate to reduce variance
            future_rewards = compute_future_rewards(rewards,gamma)
            b = future_rewards.mean(axis=0)
            A = (future_rewards - b)/future_rewards.std(axis=0)
            A = torch.from_numpy(A).double().to(device)
            
            log_probs = log_probs.double().to(device)
            # compute loss and applying gradient
            loss = torch.sum(-log_probs*A)/(num_trajectories*horizon)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            trajectory_total_rewards = rewards.sum(axis=1)
            # update scores and score_window
            scores.append(np.mean(trajectory_total_rewards))
            scores_window.append(np.mean(trajectory_total_rewards))
            
            #printing progress
            if opt_step % PRINT_EVERY == 0:
                print ("Episode: {} ... Avg reward: {:.2f}".format(opt_step,np.mean(scores_window)))
                # save the policy
                torch.save(agent.policy, 'REINFORCE-Ant.policy')
            
            if np.mean(scores_window)>= 500.0:
                print ("Environment solved in {} optimization steps! ... Avg reward : {:.2f}".format(opt_step-100,
                                                                                          np.mean(scores_window)))
                # save the policy
                torch.save(agent.policy, 'REINFORCE-Ant.policy')
                break
                
        return scores

## 4. Training the agent!

In [ ]:
# lets define and train our agent
agent = Agent(env_name='Ant-v2',learning_rate=5e-4)

In [ ]:
scores = agent.train(max_opt_steps=20000,horizon=800,gamma=0.98,PRINT_EVERY=100)

In [ ]:
# plot reward curve over episodes
plt.figure()
plt.plot(scores)
plt.xlabel('Episode #')
plt.ylabel('Total Reward')
plt.show()

## 5. Watch the smart agent!

In [ ]:
# uncomment this cell to load the trained policy for Pendulum-v0
# load policy
policy =  torch.load('REINFORCE-Ant.policy',map_location='cpu')
agent = Agent(env_name='Ant-v2')
agent.policy = policy

In [ ]:
# function to animate a list of frames
def animate_frames(frames):
    plt.figure(dpi = 72)
    plt.axis('off')

    # color option for plotting
    # use Greys for greyscale
    cmap = None if len(frames[0].shape)==3 else 'Greys'
    patch = plt.imshow(frames[0], cmap=cmap)  

    fanim = animation.FuncAnimation(plt.gcf(), \
        lambda x: patch.set_data(frames[x]), frames = len(frames), interval=30)
    
    display(display_animation(fanim, default_mode='once'))

In [ ]:
frames = []
total_reward = 0
state = env.reset()
for t in range(2000):
    action, _ = agent.policy.act(state[np.newaxis,:])
    frames.append(env.render(mode='rgb_array')) 
    next_state, reward, done, _ = env.step(action[0])
    state=next_state
    total_reward+= reward
    if done:
        break

print ("Total reward:",total_reward)
env.close()
animate_frames(frames)